<a href="https://colab.research.google.com/github/CodeMonkey01/DataMining1/blob/main/SVM_Scherb_Floris_HyperparameterOpt_min_df_0_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
df = files.upload()


In [ ]:
import pandas as pd

df = pd.read_csv('IMDB Dataset.csv')
df.describe(include='all')

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.iloc[3,0]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
%pip install tensorflow-text
import pandas as pd
from sklearn.preprocessing import LabelEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 7.1 MB/s 
     |████████████████████████████████| 578.0 MB 15 kB/s 
     |████████████████████████████████| 5.9 MB 30.9 MB/s 
     |████████████████████████████████| 1.7 MB 13.6 MB/s 
     |████████████████████████████████| 438 kB 50.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

# **Getting an overview over the Data:**

In [ ]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
df[df["sentiment"]== "positive"]

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
...,...,...
49983,"I loved it, having been a fan of the original ...",positive
49985,Imaginary Heroes is clearly the best film of t...,positive
49989,I got this one a few weeks ago and love it! It...,positive
49992,John Garfield plays a Marine who is blinded by...,positive


In [ ]:
df[df["sentiment"]== "negative"]

,review,sentiment
3,Basically there's a family where a little boy ...,negative
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
10,Phil the Alien is one of those quirky films wh...,negative
11,I saw this movie when I was about 12 when it c...,negative
...,...,...
49994,This is your typical junk comedy.<br /><br />T...,negative
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


##**Check for imbalance**

In [ ]:
df_pos = df[df["sentiment"]== "positive"]
df_neg = df[df["sentiment"]== "negative"]

#drop the difference between the positive and negative set.

df_raw_pos = df.iloc[186: , : ]
df_raw = pd.concat([df_raw_pos,df], ignore_index=True)
df_raw.sample(frac=1).reset_index(drop=True)

#Check for balance now.

print(df_raw["sentiment"].value_counts())

positive    49922
negative    49892
Name: sentiment, dtype: int64


# **Clean the reviews from HTML Tags**

Example:

In [ ]:
#16569
#view the reviews with HTML tags in the sentences.
#pd.set_option('display.max_colwidth', None)

df[df['review'].str.contains(r'<[^<>]*>') == True]


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49994,This is your typical junk comedy.<br /><br />T...,negative
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative


In [ ]:
#remove HTML Tags

df['review'] = df['review'].str.replace(r'<[^<>]*>', '', regex=True)

In [ ]:
# todo --> take (random) sample to speed up training

df_sampled = df.sample(20_000)
df = df
print(df_sampled["sentiment"].value_counts())

negative    10021
positive     9979
Name: sentiment, dtype: int64


In [ ]:
print(df["sentiment"].value_counts())

positive    25000
negative    25000
Name: sentiment, dtype: int64


# **Preprocessing**

The preprosessing for SVM consists of only stemming, since this approach appeared to show the best results. Also, the data gehts vectorized via Tf/idf.

In [ ]:
ex = df_raw.iloc[1:2]

In [ ]:
ex

,review,sentiment
1,While I count myself as a fan of the Babylon 5...,negative


In [ ]:
ex["sentiment"] = ex["sentiment"].apply(lambda x: 1 if x == "positive" else 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# Transform class from Boolean to integer value.

df_sampled['sentiment'] = df_sampled['sentiment'].apply(lambda x: 1 if x == "positive" else 0)
# df_sampled['sentiment'].apply(lambda x: 1 if x == "positive" else 0)
# df['sentiment'].apply(lambda x: 1 if x=="positive" else 0)

In [ ]:
df_sampled['sentiment'].head()

17954    0
11312    0
30645    1
32164    0
30814    1
Name: sentiment, dtype: int64

In [ ]:
df_sampled.head()

,review,sentiment
17954,This movie spends most of its time preaching t...,0
11312,Horror/Sci-Fi that is interesting as it is lau...,0
30645,Just Before Dawn is one of those really good s...,1
32164,I watched this movie recently together with my...,0
30814,Red Eye is not the kind of movie that's going ...,1


# **Stemming**

In [ ]:
import re
import nltk
from nltk.stem import PorterStemmer

token_pattern = re.compile(r"(?u)\b\w\w+\b")

ps = PorterStemmer()

nltk.download('punkt')
nltk.download('stopwords')

df_sampled['stemmed']=df_sampled['review'].apply(lambda x: ' '.join([ps.stem(y) for y in token_pattern.findall(x)]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Grid Search and Cross Validation**



**Define Target variable and redefine data for training**

In [ ]:
df_target = df_sampled['sentiment']
#df_sampled = df.copy()
#df_sampled = df_sampled.drop(columns='review')
df_sampled = df_sampled.rename({'sentiment': 'sentiment_variable'}, axis = 1)

df_sampled.head(5)

,review,sentiment_variable,stemmed
17954,This movie spends most of its time preaching t...,0,thi movi spend most of it time preach that it ...
11312,Horror/Sci-Fi that is interesting as it is lau...,0,horror sci fi that is interest as it is laugha...
30645,Just Before Dawn is one of those really good s...,1,just befor dawn is one of those realli good sl...
32164,I watched this movie recently together with my...,0,watch thi movi recent togeth with my sister wh...
30814,Red Eye is not the kind of movie that's going ...,1,red eye is not the kind of movi that go to win...


In [ ]:
df_target.head()

17954    0
11312    0
30645    1
32164    0
30814    1
Name: sentiment, dtype: int64

Vectorize stemmed_stop_removed column

In [ ]:
# Tfidf vectorizer

from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
vectorizer = TfidfVectorizer(min_df=0.001)
matrix = vectorizer.fit_transform(df_sampled['stemmed'])
df_data = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#pd.set_option('max_columns', None)
df_data.head(5)

,00,000,10,100,1000,101,11,12,13,13th,...,yup,zane,zani,zealand,zero,zip,zombi,zone,zoo,zoom
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.122477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Here we train the final model with the identified parameters and calculate the scores accuracy afterwards.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

# Create train/test split
df_data_train, df_data_test, df_target_train, df_target_test = train_test_split(
    df_data, df_target, test_size=0.2, random_state=42)

svm = LinearSVC(random_state=42,max_iter=5000)

# Specify the tunable hyper parameters
parameters = {
    'penalty': ['l2','l1'],
    'loss': ['hinge','squared_hinge'],
    'dual': [True,False],
    'tol': [1, 1e-01, 1e-02, 1e-03, 1e-04, 1e-05, 1e-06],
    'C': [1, 2, 10, 100, 1000]
}

# Define KFold parameters
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#run grid search and train model
estimator = GridSearchCV(svm, parameters, scoring="accuracy", cv=cv)
estimator.fit(df_data_train, df_target_train)

#print results
print(estimator.best_params_)
print(estimator.best_estimator_)
print(estimator.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

{'C': 1, 'dual': True, 'loss': 'hinge', 'penalty': 'l2', 'tol': 0.1}
LinearSVC(C=1, loss='hinge', max_iter=5000, random_state=42, tol=0.1)
0.8791249999999999
